In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

#读取文件
files = [
    "Team Stats Arsenal (1).xlsx",
    "Team Stats Arsenal (2).xlsx",
    "Team Stats Arsenal (3).xlsx",
    "Team Stats Arsenal (4).xlsx"
]

dfs = [pd.read_excel(f, sheet_name="TeamStats", skiprows=[1, 2]) for f in files]
df = pd.concat(dfs, ignore_index=True)

#自动命名合并列
new_cols = []
current_title = None
count = 0

#制定命名规则，以大标题命名每个小列
semantic_map = {
    "Shots / on target": ["Shots", "On_Target", "Shots_on_target_Accuracy"],
    "Passes / accurate": ["Passes", "Accurate", "Passes_accurate_Accuracy"],
    "Losses / Low / Medium / High": ["Losses", "Losses_Low", "Losses_Medium", "Losses_High"],
    "Recoveries / Low / Medium / High": ["Recoveries", "Recoveries_Low", "Recoveries_Medium", "Recoveries_High"],
    "Duels / won": ["Duels", "Won", "Duels / won_Accuracy"],
    "Shots from outside penalty area / on target": ["Shots_from_outside_penalty_area", "Shots_from_outside_penalty_area_On_Target", "Shots from outside penalty area/on target_Accuracy"],
    "Positional attacks / with shots": ["Positional_attacks", "Positional_attacks_WithShots", "Accuracy"],
    "Counterattacks / with shots": ["Counterattacks", "Counterattacks_WithShots", "Accuracy"],
    "Set pieces / with shots": ["Set_pieces", "Set_pieces_WithShots", "Set pieces/with shots_Accuracy"],
    "Corners / with shots": ["Corners", "Corners_WithShots", "Corners/with shots_Accuracy"],
    "Free kicks / with shots": ["Free_kicks", "Free_kicks_WithShots", "Free kicks/with shots_Accuracy"],
    "Penalties / converted": ["Penalties", "Converted", "Penalties/converted_Accuracy"],
    "Crosses / accurate": ["Crosses", "Crosses_Accurate", "Crosses/accurate_Accuracy"],
    "Penalty area entries (runs / crosses)": ["Penalty_area_entries", "Penalty_area_entries_Runs", "Penalty_area_entries_Crosses"],
    "Offensive duels / won": ["Offensive_duels", "Offensive duels_Won", "Offensive duels/won_Accuracy"],
    "Shots against / on target": ["Shots_against", "Shots_against_OnTarget", "Shots against/on target_Accuracy"],
    "Defensive duels / won": ["Defensive_duels", "Defensive duels_Won", "Defensive duels/won_Accuracy"],
    "Aerial duels / won": ["Aerial_duels", "Aerial duels_Won", "Aerial duels/won_Accuracy"],
    "Sliding tackles / successful": ["Sliding_tackles", "Successful", "Sliding tackles/successful_Accuracy"],
    "Forward passes / accurate": ["Forward_passes", "Forward_passes_Accurate", "Forward passes/accurate_Accuracy"],
    "Back passes / accurate": ["Back_passes", "Back_passes_Accurate", "Back passes/accurate_Accuracy"],
    "Lateral passes / accurate": ["Lateral_passes", "Lateral_passes_Accurate", "Lateral passes/accurate_Accuracy"],
    "Long passes / accurate": ["Long_passes", "Long_passes_Accurate", "Long passes/accurate_Accuracy"],
    "Passes to final third / accurate": ["Passes_to_final_third", "Passes_to_final_third_Accurate", "Accuracy"],
    "Progressive passes / accurate": ["Progressive_passes", "Progressive_passes_Accurate", "Progressive passes/accurate_Accuracy"],
    "Smart passes / accurate": ["Smart_passes", "Smart_passes_Accurate", "Smart passes/accurate_Accuracy"],
    "Throw ins / accurate": ["Throw_ins", "Throw_ins_Accurate", "Throw ins/accurate_Accuracy"],
}

for col in df.columns:
    if "Unnamed" not in str(col):
        current_title = col.strip()
        count = 0
        new_cols.append(current_title)
    else:
        count += 1
        if current_title in semantic_map:
            names = semantic_map[current_title]
            if count - 1 < len(names):
                new_cols.append(names[count - 1])
#整理列名格式
cleaned = []
for c in new_cols:
    name = re.sub(r"[ /,%]", "_", c).strip()
    name = re.sub(r"_+", "_", name)
    name = name.strip("_")
    cleaned.append(name)
df.columns = cleaned

#区分主客场
def get_home_away(match_str):
    if isinstance(match_str, str):
        if match_str.startswith("Arsenal - "):
            return "Home"
        elif " - Arsenal" in match_str:
            return "Away"
    return "Unknown"

df["HomeAway"] = df["Match"].apply(get_home_away)

#拆分阿森纳与对手
arsenal_df = df[df["Team"] == "Arsenal"].copy()
opp_df = df[df["Team"] != "Arsenal"].copy()

common_cols = list(set(arsenal_df.columns) & set(opp_df.columns))
common_cols.remove("Team")

#按Match合并
merged = pd.merge(
    arsenal_df,
    opp_df,
    on="Match",
    suffixes=("_Arsenal", "_Opponents")
)

#每90分钟进球率
if "Duration_Arsenal" in merged.columns:
    merged["Goals_per_90"] = merged["Goals_Arsenal"] / (merged["Duration_Arsenal"] / 90)
else:
    merged["Goals_per_90"] = merged["Goals_Arsenal"]
merged["Goals_per_90"] = merged["Goals_per_90"].fillna(0)
target_col = "Goals_per_90"

#文字数据转换为数字编号，跳过日期列
for col in merged.select_dtypes(include=['object']).columns:
    if "Date" not in col:
        merged[col] = LabelEncoder().fit_transform(merged[col].astype(str))

#将y作为预测目标，其余列作为特征x，避免答案泄露
x = merged.drop(
    columns=[c for c in merged.columns if "Goal" in c or "Conceded" in c or "Date" in c],
    errors="ignore")
y = merged["Goals_per_90"]

#构建模型
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(x, y)

feature_importances = pd.Series(rf.feature_importances_, index=x.columns).sort_values(ascending=False)
top20 = feature_importances.head(20)

#列出Top20
print("Top 20 Features:")
print(top20)

#特征重要性
print("特征占比总和:", feature_importances.sum())
print("Top20特征占比:", top20.sum())

Top 20 Features:
Shots_Arsenal                            0.204056
Shots_against_Opponents                  0.168276
xG_Arsenal                               0.083640
Losses_Low_Medium_High_Arsenal           0.035835
On_Target_Arsenal                        0.024866
Set_pieces_with_shots_Opponents          0.020505
Lateral_passes_Accurate_Opponents        0.013825
Shots_against_OnTarget_Opponents         0.012823
Long_passes_accurate_Arsenal             0.007974
xG_Opponents                             0.007954
Defensive_duels_Arsenal                  0.007820
Recoveries_Low_Medium_High_Opponents     0.007499
Average_shot_distance_Arsenal            0.007308
Positional_attacks_WithShots_Arsenal     0.006240
Match_tempo_Arsenal                      0.006216
Long_passes_Accurate_Opponents           0.006171
Penalty_area_entries_Opponents           0.005863
Progressive_passes_Accurate_Opponents    0.005651
Competition_Arsenal                      0.005395
Penalty_area_entries_Arsenal     

In [2]:
#测试打印日期
print("前10个 Arsenal 日期：")
print(merged["Date_Arsenal"].head(10))

print("\n前10个 Opponent 日期：")
print(merged["Date_Opponents"].head(10))

#测试提取年份
merged["Year"] = pd.to_datetime(merged["Date_Arsenal"], errors="coerce").dt.year
print("\n年份列测试：")
print(merged["Year"].unique())

#检查是否有空值
print("\n日期中无效的数量：", merged["Year"].isna().sum())

前10个 Arsenal 日期：
0    2023-05-28
1    2023-05-20
2    2023-05-20
3    2023-05-14
4    2023-05-07
5    2023-05-07
6    2023-05-02
7    2023-05-02
8    2023-04-26
9    2023-04-21
Name: Date_Arsenal, dtype: object

前10个 Opponent 日期：
0    2023-05-28
1    2023-05-20
2    2022-01-09
3    2023-05-14
4    2023-05-07
5    2021-05-02
6    2023-05-02
7    2020-12-26
8    2023-04-26
9    2023-04-21
Name: Date_Opponents, dtype: object

年份列测试：
[2023 2022 2021 2020 2019]

日期中无效的数量： 0


In [3]:
#训练模型
#确保年份存在
if "Year" not in merged.columns:
    merged["Year"] = pd.to_datetime(merged["Date_Arsenal"], errors="coerce").dt.year

#按年份计算每年平均特征
yearly = merged.groupby("Year").mean(numeric_only=True)
print("每年平均进球数:")
print(yearly[["Goals_per_90"]].head())

#提取2019–2023年的数据作为训练
train = yearly.loc[2019:2023].copy()

#以每年平均数据重新训练特征x和目标y
x_train = train.drop(columns=[c for c in train.columns if "Goal" in c or "Conceded" in c or "Date" in c], errors="ignore")
y_train = train["Goals_per_90"]

#训练模型
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(x_train, y_train)

#输出训练
print(f"\n模型训练共使用特征数：{x_train.shape[1]}")
print("训练数据年份范围：", train.index.min(), "-", train.index.max())

每年平均进球数:
      Goals_per_90
Year              
2019      1.758944
2020      1.471192
2021      1.613198
2022      1.743080
2023      1.819502

模型训练共使用特征数：211
训练数据年份范围： 2019 - 2023


In [4]:
#预测
#计算每年平均
yearly = merged.groupby("Year").mean(numeric_only=True)

#确认有哪些年份
print("参考年份：", yearly.index.tolist())

#用2023年的表现作为基础
future_features = yearly.loc[2023].to_frame().T
future_x = future_features[x_train.columns]

#预测结果
y_pred = rf.predict(future_x)
print(f"预测2023-2024的平均进球数: {y_pred[0]:.3f}")

#对比过去趋势
print("\n历史平均每90分钟比赛的进球数:")
print(yearly["Goals_per_90"])

参考年份： [2019, 2020, 2021, 2022, 2023]
预测2023-2024的平均进球数: 1.758

历史平均每90分钟比赛的进球数:
Year
2019    1.758944
2020    1.471192
2021    1.613198
2022    1.743080
2023    1.819502
Name: Goals_per_90, dtype: float64


In [6]:
#2023-2024的真实数据
files_new = ["Team Stats Arsenal.xlsx"]

dfs_new = [pd.read_excel(f, sheet_name="TeamStats", skiprows=[1, 2]) for f in files_new]
df = pd.concat(dfs_new, ignore_index=True)

#重复数据处理
new_cols = []
current_title = None
count = 0

semantic_map = {
    "Shots / on target": ["Shots", "On_Target", "Shots_on_target_Accuracy"],
    "Passes / accurate": ["Passes", "Accurate", "Passes_accurate_Accuracy"],
    "Losses / Low / Medium / High": ["Losses", "Losses_Low", "Losses_Medium", "Losses_High"],
    "Recoveries / Low / Medium / High": ["Recoveries", "Recoveries_Low", "Recoveries_Medium", "Recoveries_High"],
    "Duels / won": ["Duels", "Won", "Duels / won_Accuracy"],
    "Shots from outside penalty area / on target": ["Shots_from_outside_penalty_area", "Shots_from_outside_penalty_area_On_Target", "Shots from outside penalty area/on target_Accuracy"],
    "Positional attacks / with shots": ["Positional_attacks", "Positional_attacks_WithShots", "Accuracy"],
    "Counterattacks / with shots": ["Counterattacks", "Counterattacks_WithShots", "Accuracy"],
    "Set pieces / with shots": ["Set_pieces", "Set_pieces_WithShots", "Set pieces/with shots_Accuracy"],
    "Corners / with shots": ["Corners", "Corners_WithShots", "Corners/with shots_Accuracy"],
    "Free kicks / with shots": ["Free_kicks", "Free_kicks_WithShots", "Free kicks/with shots_Accuracy"],
    "Penalties / converted": ["Penalties", "Converted", "Penalties/converted_Accuracy"],
    "Crosses / accurate": ["Crosses", "Crosses_Accurate", "Crosses/accurate_Accuracy"],
    "Penalty area entries (runs / crosses)": ["Penalty_area_entries", "Penalty_area_entries_Runs", "Penalty_area_entries_Crosses"],
    "Offensive duels / won": ["Offensive_duels", "Offensive duels_Won", "Offensive duels/won_Accuracy"],
    "Shots against / on target": ["Shots_against", "Shots_against_OnTarget", "Shots against/on target_Accuracy"],
    "Defensive duels / won": ["Defensive_duels", "Defensive duels_Won", "Defensive duels/won_Accuracy"],
    "Aerial duels / won": ["Aerial_duels", "Aerial duels_Won", "Aerial duels/won_Accuracy"],
    "Sliding tackles / successful": ["Sliding_tackles", "Successful", "Sliding tackles/successful_Accuracy"],
    "Forward passes / accurate": ["Forward_passes", "Forward_passes_Accurate", "Forward passes/accurate_Accuracy"],
    "Back passes / accurate": ["Back_passes", "Back_passes_Accurate", "Back passes/accurate_Accuracy"],
    "Lateral passes / accurate": ["Lateral_passes", "Lateral_passes_Accurate", "Lateral passes/accurate_Accuracy"],
    "Long passes / accurate": ["Long_passes", "Long_passes_Accurate", "Long passes/accurate_Accuracy"],
    "Passes to final third / accurate": ["Passes_to_final_third", "Passes_to_final_third_Accurate", "Accuracy"],
    "Progressive passes / accurate": ["Progressive_passes", "Progressive_passes_Accurate", "Progressive passes/accurate_Accuracy"],
    "Smart passes / accurate": ["Smart_passes", "Smart_passes_Accurate", "Smart passes/accurate_Accuracy"],
    "Throw ins / accurate": ["Throw_ins", "Throw_ins_Accurate", "Throw ins/accurate_Accuracy"],
}

for col in df.columns:
    if "Unnamed" not in str(col):
        current_title = col.strip()
        count = 0
        new_cols.append(current_title)
    else:
        count += 1
        if current_title in semantic_map:
            names = semantic_map[current_title]
            if count - 1 < len(names):
                new_cols.append(names[count - 1])

cleaned = []
for c in new_cols:
    name = re.sub(r"[ /,%]", "_", c).strip()
    name = re.sub(r"_+", "_", name)
    cleaned.append(name.strip("_"))
df.columns = cleaned

df["HomeAway"] = df["Match"].apply(get_home_away)

arsenal_df = df[df["Team"] == "Arsenal"].copy()
opp_df = df[df["Team"] != "Arsenal"].copy()

merged_new = pd.merge(
    arsenal_df,
    opp_df,
    on="Match",
    suffixes=("_Arsenal", "_Opponents")
)

if "Duration_Arsenal" in merged_new.columns:
    merged_new["Goals_per_90"] = merged_new["Goals_Arsenal"] / (merged_new["Duration_Arsenal"] / 90)
else:
    merged_new["Goals_per_90"] = merged_new["Goals_Arsenal"]
merged_new["Goals_per_90"] = merged_new["Goals_per_90"].fillna(0)

for col in merged_new.select_dtypes(include=['object']).columns:
    if "Date" not in col:
        merged_new[col] = LabelEncoder().fit_transform(merged_new[col].astype(str))

merged_new["Year"] = pd.to_datetime(merged_new["Date_Arsenal"], errors="coerce").dt.year
yearly_new = merged_new.groupby("Year").mean(numeric_only=True)

#输出真实结果
print("实际 2023–2024 平均每90分钟进球数：")
print(yearly_new[["Goals_per_90"]])

#读取预测结果
predicted = y_pred[0]

#取两年平均
if 2023 in yearly_new.index and 2024 in yearly_new.index:
    real_value = yearly_new.loc[[2023, 2024], "Goals_per_90"].mean()
elif 2024 in yearly_new.index:
    real_value = yearly_new.loc[2024, "Goals_per_90"]
else:
    real_value = yearly_new.loc[2023, "Goals_per_90"]

#计算误差
error = abs(predicted - real_value)
print(f"\n模型预测: {predicted:.3f}")
print(f"实际结果（2023–2024平均）: {real_value:.3f}")
print(f"绝对误差: {error:.3f}")

实际 2023–2024 平均每90分钟进球数：
      Goals_per_90
Year              
2023      1.682155
2024      2.212079

模型预测: 1.758
实际结果（2023–2024平均）: 1.947
绝对误差: 0.189
